In [18]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import pandas as pd
import time
import csv

In [22]:
%pip install requests beautifulsoup4 selenium pandas

Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install chromedriver

In [24]:
# 셀레니움 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 백그라운드에서 실행되도록 설정
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 크롬 드라이버 실행 경로 설정 (실제 경로로 변경 필요)
driver = webdriver.Chrome(options=chrome_options)

# Watcha 사이트 열기
url = 'https://pedia.watcha.com/ko-KR'  # Watcha 사이트 URL (또는 크롤링할 사이트 URL)
driver.get(url)

time.sleep(5)  # 페이지 로딩 대기


In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

movies_data = []

# 영화 리스트 추출
movies = soup.find_all('li', class_='_DV31')

# 전체 순위를 추적할 변수
overall_rank = 1

for movie in movies[:30]:  # 1위부터 30위까지만 크롤링
    # 영화 제목
    title = movie.find('div', class_='ZADAQ x_ja6')
    title_text = title.text if title else 'N/A'
    
    # 영화 순위
    rank = movie.find('div', class_='pfQ5u IltnH XUMRY')
    rank_text = rank.text if rank else str(overall_rank)  # 순위가 없으면 순서대로 채움
    
    # 예매율 및 누적 관객 수 추출
    booking_info = movie.find('div', class_='ycnKW G41oJ')
    if booking_info:
        booking_info_text = booking_info.text
        booking_rate = booking_info_text.split(' ・ ')[0].replace("예매율 ", "")
        audience_count = booking_info_text.split(' ・ ')[1].replace("누적 관객 ", "") if ' ・ ' in booking_info_text else 'N/A'
    else:
        booking_rate = 'N/A'
        audience_count = 'N/A'
    
    # 영화 이미지 URL 추출
    image = movie.find('img', class_='rHgxN BuYr5')
    image_url = image['src'] if image else 'N/A'
    
    # 영화 세부 정보 저장
    movies_data.append({
        '순위': rank_text,
        '제목': title_text,
        '예매율': booking_rate,
        '누적 관객 수': audience_count,
        '이미지 URL': image_url
    })
    
    # 전체 순위 증가
    overall_rank += 1

# 데이터를 데이터프레임으로 변환
df = pd.DataFrame(movies_data)

# 순위가 'N/A'인 항목을 제거하고, 1위부터 30위까지만 출력
df_clean = df[df['순위'] != 'N/A'].head(30)

# 데이터 출력 또는 CSV 파일로 저장
df_clean.to_csv('boxoffice_movie_data.csv', index=False, encoding='utf-8-sig')
print(df_clean)

# 드라이버 종료
driver.quit()

KeyError: '순위'